### Script to determine the time when sediment transport was captured by hydrophones in La Jara

In [2]:
import os
import re
import numpy as np
import librosa
import scipy.signal
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


Defining functions

In [ ]:
def plot_raw_signals(storm_folder):
    # create output folder
    output_folder = os.path.join(storm_folder, "raw_signal_plots")
    os.makedirs(output_folder, exist_ok=True)

    # Initialize lists to store min/max values of all signals
    all_left_signals = []
    all_right_signals = []

    # First loop through all files to collect min/max values
    for filename in sorted(os.listdir(storm_folder)):
        if filename.endswith(".flac"):
            filepath = os.path.join(storm_folder, filename)
            try:
                # Load stereo signal
                signal, sr = librosa.load(filepath, sr=None, mono=False)

                # Append the signals to the lists
                all_left_signals.append(signal[0])
                all_right_signals.append(signal[1])

            except Exception as e:
                print(f"Skipped {filename}: {e}")

    # Now calculate global min/max for left and right signals across all files
    left_min = np.min(np.concatenate(all_left_signals))
    left_max = np.max(np.concatenate(all_left_signals))
    right_min = np.min(np.concatenate(all_right_signals))
    right_max = np.max(np.concatenate(all_right_signals))

    # Loop again to plot the signals with consistent Y-axis limits
    for filename in sorted(os.listdir(storm_folder)):
        if filename.endswith(".flac"):
            filepath = os.path.join(storm_folder, filename)
            try:
                # Load stereo signal
                signal, sr = librosa.load(filepath, sr=None, mono=False)

                # Create time axis
                duration = signal.shape[-1] / sr
                time = np.linspace(0, duration, signal.shape[-1])

                # Plot
                fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

                # Plot Right Channel (Top)
                axes[0].plot(time, signal[1], label='Right Channel', color='b', alpha=0.7)
                axes[0].set_title(f"Right Channel: {filename}")
                axes[0].set_ylabel("Amplitude")
                axes[0].legend()

                # Plot Left Channel (Bottom)
                axes[1].plot(time, signal[0], label='Left Channel', color='r', alpha=0.7)
                axes[1].set_title(f"Left Channel: {filename}")
                axes[1].set_xlabel("Time (s)")
                axes[1].set_ylabel("Amplitude")
                axes[1].legend()

                # Set consistent Y-axis limits
                axes[0].set_ylim([right_min, right_max])  # Right channel
                axes[1].set_ylim([left_min, left_max])  # Left channel

                # Adjust layout to prevent overlap
                plt.tight_layout()

                # Save the figure
                save_path = os.path.join(output_folder, f"{filename.replace('.flac', '')}_raw.png")
                plt.savefig(save_path)
                plt.close()

            except Exception as e:
                print(f"Skipped {filename}: {e}")



############################################################

def compute_baseflow_threshold(baseflow_folder, channel="left", margin_factor=1):
    envelopes = []

    for filename in sorted(os.listdir(baseflow_folder)):
        if filename.endswith(".flac"):
            filepath = os.path.join(baseflow_folder, filename) # get full path
            # parse timestamp from filename
            try:
                signal, sr = librosa.load(filepath, sr=None, mono=False) # load stereo audio
                if signal.ndim > 1:
                    signal = signal[0] if channel == "left" else signal[1] # select channel input by user

                # compute the envelope using the Hilbert transform
                analytic_signal = scipy.signal.hilbert(signal)
                envelope = np.abs(analytic_signal)
                envelopes.append(envelope)
            except Exception as e:
                print(f"Skipping {filename}: {e}")

    # Concatenate all envelopes and find the maximum value
    all_envelopes = np.concatenate(envelopes)
    max_envelope_value = np.max(all_envelopes)
    envelope_std = np.std(all_envelopes) # calculate std deviation of all envelopes
    error_margin = margin_factor * envelope_std

    # Set the threshold as the max envelope value + the error margin
    threshold = max_envelope_value + error_margin

    print(f"Computed threshold (max + error): {threshold:.6f} (max: {max_envelope_value:.6f}, margin: {error_margin:.6f})")
    return threshold

#############################################################

def get_first_impact_time(storm_folder, threshold, channel='left'):
    impact_times = []
    envelope_folder = os.path.join(storm_folder, "envelopes")
    os.makedirs(envelope_folder, exist_ok=True)

    for filename in sorted(os.listdir(storm_folder)): # lopp thru all .flac files
        if filename.endswith('.flac'):
            # parse timestamp from filename
            match = re.search(r'(\d{6})-(\d{6})', filename) # in MMDDYY-HHMMSS format
            if not match: # if none found, skip this file
                continue
            date_part = match.group(1)
            time_part = match.group(2)
            start_time = datetime.strptime(date_part + time_part, '%m%d%y%H%M%S') # create date time object with that time

            # load stereo audio (sr=None preserves original sampling rate)
            filepath = os.path.join(storm_folder, filename)
            signal, sr = librosa.load(filepath, sr=None, mono=False)
            
            # select left or right channel
            signal = signal[0] if channel == 'left' else signal[1]

            # Hilbert transform to get the envelope
            envelope = np.abs(scipy.signal.hilbert(signal))

            # plot and save envelope
            times = np.arange(len(envelope)) / sr
            plt.figure(figsize=(10, 4))
            plt.plot(times, envelope, label="Envelope")
            plt.axhline(y=threshold, color='r', linestyle='--', label='Threshold')
            plt.title(f"Envelope: {filename}")
            plt.xlabel("Time (s)")
            plt.ylabel("Normalized Amplitude")
            plt.legend()
            plt.tight_layout()

            save_path = os.path.join(envelope_folder, f"{filename.replace('.flac', '')}_envelope.png")
            plt.savefig(save_path)
            plt.close()
            
            # Find when the signal exceeds the threshold
            above_threshold = np.where(envelope > threshold)[0]
            if len(above_threshold) > 0:
                impact_sample = above_threshold[0]
                impact_time = start_time + timedelta(seconds=impact_sample / sr)
                impact_times.append((filename, impact_time))
                print(f"First impact in {filename} at {impact_time}")
                break  # stop after first impact is found

    return impact_times

Plotting the raw acoustic signal

In [15]:
storm_path = "H1/st3"
plot_raw_signals(storm_path) 

In [16]:
storm_path = "H1/st4"
plot_raw_signals(storm_path) 

In [17]:
storm_path = "H1/st5"
plot_raw_signals(storm_path) 

In [18]:
storm_path = "H1/st6"
plot_raw_signals(storm_path) 

In [19]:
storm_path = "H1/st7"
plot_raw_signals(storm_path) 

Processing the timing of each storm: 

In [ ]:
storm_path = "H1/st1"
baseflow_path = "H1/st1/baseflow"

threshold = compute_baseflow_threshold(baseflow_path, channel="right")
get_first_impact_time(storm_path, threshold, channel="right")

In [13]:
storm_path = "H1/st2"
baseflow_path = "H1/st2/baseflow"

threshold = compute_baseflow_threshold(baseflow_path, channel="right")
get_first_impact_time(storm_path, threshold, channel="right")

Computed threshold (max + error): 0.009335 (max: 0.009218, margin: 0.000118)
First impact in raspberrypi_080322-143100.flac at 2022-08-03 14:31:42.371927


[('raspberrypi_080322-143100.flac',
  datetime.datetime(2022, 8, 3, 14, 31, 42, 371927))]

In [20]:
storm_path = "H1/st3"
baseflow_path = "H1/st3/baseflow"

threshold = compute_baseflow_threshold(baseflow_path, channel="left")
get_first_impact_time(storm_path, threshold, channel="left")

Computed threshold (max + error): 2.887115 (max: 2.872973, margin: 0.014142)


[]

In [21]:
storm_path = "H1/st4"
baseflow_path = "H1/st4/baseflow"

threshold = compute_baseflow_threshold(baseflow_path, channel="left")
get_first_impact_time(storm_path, threshold, channel="left")

Computed threshold (max + error): 0.005599 (max: 0.005481, margin: 0.000119)
First impact in raspberrypi_072923-133000.flac at 2023-07-29 13:30:14.182268


[('raspberrypi_072923-133000.flac',
  datetime.datetime(2023, 7, 29, 13, 30, 14, 182268))]

In [22]:
storm_path = "H1/st5"
baseflow_path = "H1/st5/baseflow"

threshold = compute_baseflow_threshold(baseflow_path, channel="right")
get_first_impact_time(storm_path, threshold, channel="right")

Computed threshold (max + error): 0.177341 (max: 0.173669, margin: 0.003672)
First impact in raspberrypi_081323-183000.flac at 2023-08-13 18:30:00.018707


[('raspberrypi_081323-183000.flac',
  datetime.datetime(2023, 8, 13, 18, 30, 0, 18707))]

In [23]:
storm_path = "H1/st6"
baseflow_path = "H1/st6/baseflow"

threshold = compute_baseflow_threshold(baseflow_path, channel="left")
get_first_impact_time(storm_path, threshold, channel="left")

Computed threshold (max + error): 0.021649 (max: 0.021173, margin: 0.000476)
First impact in raspberrypi_082823-121500.flac at 2023-08-28 12:15:30.017868


[('raspberrypi_082823-121500.flac',
  datetime.datetime(2023, 8, 28, 12, 15, 30, 17868))]

In [24]:
storm_path = "H1/st7"
baseflow_path = "H1/st7/baseflow"

threshold = compute_baseflow_threshold(baseflow_path, channel="left")
get_first_impact_time(storm_path, threshold, channel="left")

Computed threshold (max + error): 0.012496 (max: 0.012273, margin: 0.000223)


C:\Users\huck4481\AppData\Local\Temp\ipykernel_79180\1227782215.py:145: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()
C:\Users\huck4481\AppData\Local\Temp\ipykernel_79180\1227782215.py:148: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(save_path)


First impact in raspberrypi_091423-140000.flac at 2023-09-14 14:00:01.836757


[('raspberrypi_091423-140000.flac',
  datetime.datetime(2023, 9, 14, 14, 0, 1, 836757))]